In [0]:
%sql
SELECT * FROM `hive_metastore`.`default`.`ice_data_2025_02_28_135500_2_csv` where is_dupe = "False";

In [0]:

import pandas as pd


df = spark.table("hive_metastore.default.ice_data_2025_02_28_135500_2_csv").toPandas()
df.head()
df_original = df

# filter to just is_dupe = False
df = df[df['is_dupe'] == 'False']

df.info()


In [0]:
# load the airports GIS data
df_airports = spark.table("default.international_airports_lower_48").toPandas()
df_airports.head()

In [0]:
from math import radians, cos, sin, asin, sqrt

# add a haversine distance formula
def haversine(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371
    # return distance in miles
    miles = c * r * 0.621371
    return miles

# find the closest airport for each ice lead
def find_closest_airport(row):
    min_distance = float('inf')
    closest_airport = None
    for _, airport in df_airports.iterrows():
        distance = haversine(float(row['longitude']), float(row['latitude']), float(airport['Longitude']), float(airport['Latitude']))
        if distance < min_distance:
            min_distance = distance
            closest_airport = airport['Airport Name']
    return pd.Series([closest_airport, min_distance], index=['closest_airport', 'distance_to_airport'])

df[['closest_airport', 'distance_to_airport']] = df.apply(find_closest_airport, axis=1)

In [0]:

df.head()

# For verfication, select just the lead_id, address, closest airport, and distance for rows with an address
verification_df = df[['lead_id', 'last_known_address', 'closest_airport', 'distance_to_airport']][df['last_known_address'].notnull()]

# show the top 10 leads closest to an airport
closest_df = verification_df.sort_values(by='distance_to_airport')
display(closest_df.head(10))

# show the top 10 furthest leads from an airport
furthest_df = verification_df.sort_values(by='distance_to_airport', ascending=False)
display(furthest_df.head(10))






In [0]:
# load the flights table into a df
df_flights = spark.table("default.test_flight_data").toPandas()

display(df_flights)



display(df_flights.groupby('Destination')['Available Seats'].sum().sort_values(ascending=False))

import matplotlib.pyplot as plt
# Create a plot for each Destination and show a distribution of the available seats
for destination, group in df_flights.groupby('Destination'):
    plt.figure(figsize=(10, 6))
    plt.hist(group['Available Seats'], bins=20, edgecolor='k', alpha=0.7)
    plt.title(f'Distribution of Available Seats for {destination}')
    plt.xlabel('Available Seats')
    plt.ylabel('Frequency')
    
    # Calculate quartiles
    q1 = group['Available Seats'].quantile(0.25)
    q2 = group['Available Seats'].quantile(0.5)
    q3 = group['Available Seats'].quantile(0.75)
    
    # Add quartile markings
    plt.axvline(q1, color='r', linestyle='dashed', linewidth=1, label='Q1')
    plt.axvline(q2, color='r', linestyle='dashed', linewidth=1, label='Q2')
    plt.axvline(q3, color='r', linestyle='dashed', linewidth=1, label='Q3')
    plt.legend()
    plt.show()
    




In [0]:
# create some weighted prioritization functions

# lead needs to have an address
def address_weight(row):
    return 5 if pd.notnull(row['last_known_address']) else 0

def risk_weight(row):
    if row['risk_level'] == 'High':
        return 5
    if row['risk_level'] == 'Medium':
        return 3
    return 1

def airport_weight(row):
    if row['distance_to_airport'] < 100:
        return 5
    if row['distance_to_airport'] < 250:
        return 2
    return 0

def available_seats_for_destination_country_for_closest_airport_weight(row):
    destination_country = row['country_of_origin']
    closest_airport = row['closest_airport']
    
    # Filter flights to the closest airport and destination country
    relevant_flights = df_flights[(df_flights['Destination'] == destination_country) & 
                                  (df_flights['Airport Name'] == closest_airport)]
    
    # Sum available seats
    total_available_seats = relevant_flights['Available Seats'].sum()
    
    # Assign weight based on available seats
    if total_available_seats > 30:
        return 5
    elif total_available_seats > 20:
        return 3
    elif total_available_seats > 10:
        return 1
    else:
        return 0


def organized_crime_weight(row):
    # if pd null return 0, otherwise 5
    if pd.notnull(row['organized_crime_links']):
        return 5
    return 0

def country_of_origin_is_central_america(row): 
    # Mexico, Guatemela, Hondurus deportation agreements 
    if row['country_of_origin'] in ['Mexico', 'Guatemala', 'Honduras']:
        return 5
    if row['country_of_origin'] == 'India':
        return 1
    return 0
    

def prioritize(row):
    return (address_weight(row) + risk_weight(row) + airport_weight(row) + organized_crime_weight(row) + country_of_origin_is_central_america(row)) + available_seats_for_destination_country_for_closest_airport_weight(row)

# add a "lead_prioritization number" column with the weighted score separate from the lead prioritization field
df['lead_prioritization_number'] = df.apply(prioritize, axis=1)

# create a validation df with the columns used for weighting, the lead id, and the weighted score
verification_df = df[['lead_prioritization_number', 'lead_id', 'last_known_address', 'risk_level', 'distance_to_airport', 'organized_crime_links', 'country_of_origin' ]]

# filter to the top 5 scores and diplay
display(verification_df.sort_values(by='lead_prioritization_number', ascending=False).head(5))

# filter to the bottom 5 scores and diplay
display(verification_df.sort_values(by='lead_prioritization_number').head(5))


In [0]:
import matplotlib.pyplot as plt
import numpy as np

# Calculate quartiles
quartiles = np.percentile(df['lead_prioritization_number'], [25, 50, 75])

# Create a figure and axis
plt.figure(figsize=(10, 6))

# Plot histogram
plt.hist(df['lead_prioritization_number'], bins=20, edgecolor='k', alpha=0.7)

# Add vertical lines for quartiles
for quartile in quartiles:
    plt.axvline(quartile, color='r', linestyle='dashed', linewidth=1)

# Add text for quartiles
for i, quartile in enumerate(quartiles):
    plt.text(quartile, plt.ylim()[1] * 0.9, f'Q{i+1}', color='r', ha='center')

# Add titles and labels
plt.title('Distribution of Lead Prioritization Scores with Quartiles')
plt.xlabel('Lead Prioritization Score')
plt.ylabel('Frequency')
plt.grid(True)

# Show plot
plt.show()

In [0]:
%python
# count unique lead_ids
rows, columns  = df_original.shape

def include_risk(row):
    if row['risk_level'] == 'High':
        return True
    if row['risk_level'] == 'Medium':
        return True
    return False

def include_organized_crime(row):
    if row['organized_crime_links'] is not None:
        return True
    return False

def include_close_to_airport(row):
    if row['distance_to_airport'] < 100:
        return True
    return False

key_info = {}
data = "Total ⬛️ "
key_info[0] = {"data": data , "count": rows}

# filter is_dupe = False
rows, columns = df_original[df_original['is_dupe'] == "False"].shape   
data = "Without duplicates 🔀"
key_info[1] = {"data": data , "count": rows}

# use the include risk function to filter rows
rows, columns = df_original[
    (df_original['is_dupe'] == "False") & 
    (df_original.apply(include_risk, axis=1))
].shape
data = "High/Medium risk 🚩"
key_info[2] = {"data": data , "count": rows}

# filter by organized crime
rows, columns = df_original[
    (df_original['is_dupe'] == "False") & 
    (df_original.apply(include_organized_crime, axis=1))
].shape
data = "Organized crime ⚖️"
key_info[3] = {"data": data , "count": rows}

# filter by close to airport
rows, columns = df[
    (df['is_dupe'] == "False") & 
    (df.apply(include_close_to_airport, axis=1))
].shape
data = "Close to airport ✈️"
key_info[4] = {"data": data , "count": rows}

# filter is dupe = false, shorter flights based on country of origin is Mexico, Guatamala, Hondurus
rows, columns = df_original[
    (df_original['is_dupe'] == "False") & 
    (df_original['country_of_origin'].isin(['Mexico', 'Guatemala', 'Honduras']))
].shape
data = "Shorter flights 🌎"
key_info[6] = {"data": data , "count": rows}

# convert key info into a pandas df where the key is th
df_key_info = pd.DataFrame.from_dict(key_info, orient='index', columns=['data', "count" ])
# add thousands separator to count
df_key_info['count'] = df_key_info['count'].apply(lambda x: "{:,}".format(x))

display(df_key_info)

In [0]:
# update the lead_prioritization field to be High is score > 18, Medium if 18 >= score > 15, and Low if score <= 15
df['lead_prioritization'] = df['lead_prioritization_number'].apply(lambda x: 'High' if x > 18 else ('Medium' if x > 15 else 'Low'))


# show a pie chart of the lead_prioritization field use a color scheme where High = red, Medium = yellow, and Low = green
colors = {'High': 'red', 'Medium': 'yellow', 'Low': 'green'}
plt.figure(figsize=(6, 6))
plt.pie(df['lead_prioritization'].value_counts(), labels=df['lead_prioritization'].value_counts().index, colors=df['lead_prioritization'].map(colors))
plt.title('Lead Prioritization Distribution')
plt.show()

 
